In [ ]:
# Retail Sales Analytics – India 2023

# Imports and load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

%matplotlib inline

plt.style.use("seaborn-v0_8")
sns.set_palette("Set2")
project_root = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
data_path = project_root / "data" / "retail_sales_2023_india.csv"
df = pd.read_csv(data_path)
df.head()

# Basic info

df.info()
df.describe(include="all")
df.columns = [c.strip().lower() for c in df.columns]
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.dropna(subset=["date", "store", "category", "product", "quantity", "unit_price", "revenue"])
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["month_name"] = df["date"].dt.strftime("%b")
df["day_of_week"] = df["date"].dt.day_name()
df["calculated_revenue"] = df["quantity"] * df["unit_price"]
df["revenue_mismatch_flag"] = (df["revenue"].round(2) != df["calculated_revenue"].round(2))
df.head()

# Revenue mismatch check

df["revenue_mismatch_flag"].value_counts(normalize=True) * 100

# KPIs

total_revenue = df["revenue"].sum()
total_transactions = df.shape[0]
avg_order_value = total_revenue / total_transactions
print(f"Total Revenue: ₹{total_revenue:,.0f}")
print(f"Total Transactions: {total_transactions:,}")
print(f"Average Order Value: ₹{avg_order_value:,.2f}")

# Revenue by store

rev_by_store = (
    df.groupby("store", as_index=False)["revenue"]
    .sum()
    .sort_values("revenue", ascending=False)
)
plt.figure(figsize=(8, 4))
sns.barplot(data=rev_by_store, x="store", y="revenue")
plt.title("Total Revenue by Store")
plt.ylabel("Revenue (₹)")
plt.xlabel("Store")
plt.tight_layout()
plt.show()
rev_by_store

# Revenue by category

rev_by_cat = (
    df.groupby("category", as_index=False)["revenue"]
    .sum()
    .sort_values("revenue", ascending=False)
)

plt.figure(figsize=(8, 4))
sns.barplot(data=rev_by_cat, x="category", y="revenue")
plt.title("Total Revenue by Category")
plt.ylabel("Revenue (₹)")
plt.xlabel("Category")
plt.tight_layout()
plt.show()

rev_by_cat

# Monthly revenue trend

monthly_rev = (
    df.groupby(["year", "month", "month_name"], as_index=False)["revenue"]
    .sum()
    .sort_values(["year", "month"])
)

plt.figure(figsize=(10, 4))
sns.lineplot(data=monthly_rev, x="month_name", y="revenue", marker="o")
plt.title("Monthly Revenue Trend – 2023")
plt.ylabel("Revenue (₹)")
plt.xlabel("Month")
plt.tight_layout()
plt.show()

monthly_rev

# Store vs category heatmap

pivot_store_cat = (
    df.pivot_table(
        index="store",
        columns="category",
        values="revenue",
        aggfunc="sum"
    )
    .fillna(0)
)

plt.figure(figsize=(8, 5))
sns.heatmap(pivot_store_cat, annot=False, cmap="Blues")
plt.title("Revenue by Store and Category")
plt.ylabel("Store")
plt.xlabel("Category")
plt.tight_layout()
plt.show()

pivot_store_cat

# Weekday vs weekend

df["is_weekend"] = df["day_of_week"].isin(["Saturday", "Sunday"])

weekday_rev = (
    df.groupby("is_weekend", as_index=False)["revenue"]
    .sum()
)

weekday_rev["day_type"] = weekday_rev["is_weekend"].map({True: "Weekend", False: "Weekday"})

plt.figure(figsize=(6, 4))
sns.barplot(data=weekday_rev, x="day_type", y="revenue")
plt.title("Weekday vs Weekend Revenue")
plt.ylabel("Revenue (₹)")
plt.xlabel("")
plt.tight_layout()
plt.show()

weekday_rev

#Top products

top_products = (
    df.groupby("product", as_index=False)["revenue"]
    .sum()
    .sort_values("revenue", ascending=False)
    .head(10)
)

plt.figure(figsize=(8, 4))
sns.barplot(data=top_products, x="product", y="revenue")
plt.title("Top 10 Products by Revenue")
plt.ylabel("Revenue (₹)")
plt.xlabel("Product")
plt.tight_layout()
plt.show()

top_products
